In [1]:
# glove300
import numpy as np
import pandas as pd

import os

# 1. load dataset
# data 

data_train_glove300=pd.read_csv('data/train_glove300.csv')

data_test_glove300=pd.read_csv('data/test_glove300.csv')

data_validation_glove300=pd.read_csv('data/dev_glove300.csv')


data_train_glove300.head()

,region,user,tweet
0,NORTHEAST,USER_6197f95d,0.16529499739408493 0.15692149847745895 0.1295...
1,NORTHEAST,USER_6197f95d,0.12859000265598297 0.2504599988460541 -0.5546...
2,NORTHEAST,USER_6197f95d,-0.20041219741106034 0.3086455903947353 -0.041...
3,NORTHEAST,USER_6197f95d,-0.14732199907302856 0.3644040012732148 0.1265...
4,NORTHEAST,USER_6197f95d,0.07860666265090306 -0.19449666266640028 -0.25...


In [2]:
train_x, train_y = data_train_glove300['tweet'].copy(),data_train_glove300['region'].copy()#data_train_glove300.iloc[:, [1,2]].copy(),data_train_glove300['region'].copy()
test_x = data_test_glove300['tweet'].copy()
validation_x,validation_y = data_validation_glove300['tweet'].copy(),data_validation_glove300['region'].copy()

train_x.head()

0    0.16529499739408493 0.15692149847745895 0.1295...
1    0.12859000265598297 0.2504599988460541 -0.5546...
2    -0.20041219741106034 0.3086455903947353 -0.041...
3    -0.14732199907302856 0.3644040012732148 0.1265...
4    0.07860666265090306 -0.19449666266640028 -0.25...
Name: tweet, dtype: object

In [3]:
tweet_list = []
for i in range(len(train_x[0].split(' '))):
    temp = 'tweet'+str(i)
    tweet_list.append(temp)
train_x_300 = pd.DataFrame()
train_x_300[tweet_list] = train_x.str.split(pat=" ", expand=True).astype('float64')
#train_x_300.drop('tweet', axis=1,inplace = True)

validation_x_300 = pd.DataFrame()
validation_x_300[tweet_list] = validation_x.str.split(pat=" ", expand=True).astype('float64')
#validation_x_300.drop('tweet', axis=1,inplace = True)


test_x_300 = pd.DataFrame()
test_x_300[tweet_list] = test_x.str.split(pat=" ", expand=True).astype('float64')
#test_x_300.drop('tweet', axis=1,inplace = True)

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_y=le.fit_transform(train_y)
validation_y=le.transform(validation_y)

In [5]:
train_x_300.head()

,tweet0,tweet1,tweet2,tweet3,tweet4,tweet5,tweet6,tweet7,tweet8,tweet9,...,tweet290,tweet291,tweet292,tweet293,tweet294,tweet295,tweet296,tweet297,tweet298,tweet299
0,0.165295,0.156921,0.129570,0.126863,0.109545,0.025270,-0.155725,0.154055,-0.090687,-0.608515,...,0.172670,0.126226,-0.098614,-0.280390,0.025885,-0.072000,0.070441,-0.102425,-0.083425,-0.041377
1,0.128590,0.250460,-0.554680,0.100830,-0.122860,0.086751,-0.089401,0.186030,0.006967,-1.752000,...,-0.049889,-0.172140,0.295880,-0.082773,0.267030,-0.350290,-0.088684,-0.329630,0.258420,0.301360
2,-0.200412,0.308646,-0.041178,0.044302,0.010667,0.147315,-0.228232,0.062656,0.193391,-0.841962,...,-0.145189,-0.107838,-0.073095,0.014139,0.092654,0.020315,0.066396,-0.224434,-0.095180,0.128492
3,-0.147322,0.364404,0.126528,-0.015661,0.041453,0.039504,-0.158922,-0.054634,0.137292,-0.685044,...,0.128150,0.019823,-0.132730,0.013922,0.037354,-0.402442,0.087009,-0.081570,-0.120145,-0.134394
4,0.078607,-0.194497,-0.253428,-0.011151,-0.282250,0.121172,-0.100056,0.102113,0.007096,-0.395342,...,0.073213,0.030286,-0.169902,-0.178539,0.193187,-0.049336,0.098703,0.090435,0.150210,0.232513


In [6]:
train_x_300.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133795 entries, 0 to 133794
Columns: 300 entries, tweet0 to tweet299
dtypes: float64(300)
memory usage: 306.2 MB


In [7]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
# feature selection

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_x_300, train_y)
selectionmodel = SelectFromModel(lsvc, prefit=True,threshold=-np.inf, max_features = 100)

train_x_new = selectionmodel.transform(train_x_300)
validation_x_new = selectionmodel.transform(validation_x_300)
test_x_new = selectionmodel.transform(test_x_300)



In [8]:
train_x_new.shape

(133795, 100)

In [9]:
train_x_new_df =  pd.DataFrame(train_x_new)
validation_x_new_df = pd.DataFrame(validation_x_new)
test_x_new_df = pd.DataFrame(test_x_new)

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

LR_train = LogisticRegression()
LR_train.fit(train_x_new_df,train_y)

LR_predict_validation = LR_train.predict(validation_x_new_df)
print(accuracy_score(validation_y,LR_predict_validation))
print(f1_score(validation_y,LR_predict_validation,average='macro'))

C:\Users\49323\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\49323\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.4335511982570806
0.24926053850309787


C:\Users\49323\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [93]:
from sklearn.neural_network import MLPClassifier

mlp_train = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

mlp_train.fit(train_x_new_df,train_y)

mlp_predict_validation = mlp_train.predict(validation_x_new_df)
print(accuracy_score(validation_y,mlp_predict_validation))
print(f1_score(validation_y,mlp_predict_validation,average='macro'))

0.44034858387799564
0.25244227003650876


C:\Users\49323\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [97]:
train_x['tweet'] = train_x.str.split(pat=" ", expand=True).astype('float64')
train_x.head()
#test_x = data_test_glove300['tweet'].copy()
#validation_x,validation_y = data_validation_glove300['tweet'].copy(),data_validation_glove300['region'].copy()

0    0.16529499739408493 0.15692149847745895 0.1295...
1    0.12859000265598297 0.2504599988460541 -0.5546...
2    -0.20041219741106034 0.3086455903947353 -0.041...
3    -0.14732199907302856 0.3644040012732148 0.1265...
4    0.07860666265090306 -0.19449666266640028 -0.25...
Name: tweet, dtype: object

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn_train = KNeighborsClassifier(n_neighbors = 5)
knn_train.fit(train_x_300,train_y)
knn_predict_validation = knn_train.predict(validation_x_300)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print(accuracy_score(validation_y,knn_predict_validation))
print(f1_score(validation_y,knn_predict_validation,average='macro'))